In [1]:
from netCDF4 import Dataset as NetCDFFile 
import numpy as np
import os

In [2]:
import pandas as pd
import plotly.express as px


def display_map_sum(var_name1, var_name2,  day, dark=True, day2=None, daySum=None, difference = False, colourScale=None, Rad=3, invert=False, outputFolder = 'C:\CWatM_output', outputFolder2 = 'C:\CWatM_output', colourRange=None, compare=None):

  # The function displays the map of *var_name* on *day*.
  # If day2 != None, this displayed the difference between *day* and *day2*.
  print('testing')
  print('difference', difference)
  print(var_name1)
  print('hello hello')
  print(var_name2)
  for var_name in [var_name1, var_name2]:
        
    if 'month' not in var_name and 'annual' not in var_name :
    #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
      nc = NetCDFFile(outputFolder + '/'+var_name +'_daily.nc')
    else:
      nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')

    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    time = nc.variables['time'][:]

    if day2 != None:
      print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
      t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
    elif daySum != None: 
      print('Showing the sum of '+var_name+ ' over the simulation period. Unit are in metres (depth).')
      t2 = nc.variables[var_name][0,:,:]
      #t2 = nc.variables[var_name][day,:,:]
      #for d in range(day, daySum):
      print('number of days or months', len(nc.variables[var_name][:,0,0]))
      for d in range(len(nc.variables[var_name][:,0,0])):
        t2 += nc.variables[var_name][d,:,:]
    else:
      t2 = nc.variables[var_name][day,:,:]
    
    df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
    for i in range(len(t2[:,0])):
      for j in range(len(t2[0,:])):

        if t2[i,j]!=1200:
          df['Latitude'].append(lat[i])
          df['Longitude'].append(lon[j])

          if var_name == 'head_development' and invert == True:
            df[var_name].append(100-t2[i,j])
          else:
            #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
            df[var_name].append(t2[i,j])

          #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
          #df[var_name].append(t2[i,j]) #Notice the minimum
          df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
          #df_yield['Yield'].append(t2[i,j])
    print('in difference1')
    if var_name==var_name1:
        difference = df[var_name].copy()
     
    elif difference == False:
        difference = [difference[i] +  df[var_name][i] for i in range(len(difference))]
    
    elif difference == True:
        difference = [difference[i] - df[var_name][i] for i in range(len(difference))]

  df['difference'] = difference
  if compare==True:
    var_name = 'difference'


  if colourScale == None:
    colourScale="YlGnBu"

  if var_name == 'discharge_monthtot':
    #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
 
  else:
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z='difference', radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=7, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

  # Bhima zoom = 6.9
  fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

  fig.update_layout(
      title_text = var_name,
      margin=dict(l=5, r=5, t=30, b=10),
      paper_bgcolor='rgba(0,0,0,0)',
      font=dict( color="white"))

  if dark != True:
      fig.update_layout(
          font=dict( color="black"))

  fig.show()

In [3]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map_sum_differentRuns(var_name,  day, dark=True, day2=None, daySum=None, calcDifference = False, colourScale=None, Rad=3, invert=False, outputFolder = 'C:\CWatM_output', outputFolder2 = 'C:\CWatM_output_Examples\CWatM_output_oneYear_reduceIrrigation', colourRange=None, compare=None):

  # The function displays the map of *var_name* on *day*.
  # If day2 != None, this displayed the difference between *day* and *day2*.
  print('testing')
  #print('difference', difference)
  print(var_name)

  for output in [outputFolder, outputFolder2]:
        
    if 'month' not in var_name and 'annual' not in var_name :
    #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
      nc = NetCDFFile(output + '/'+var_name +'_daily.nc')
    else:
      nc = NetCDFFile(output + '/'+var_name + '.nc')

    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    time = nc.variables['time'][:]

    if day2 != None:
      print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
      t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
    elif daySum != None: 
      print('Showing the sum of '+var_name+ ' over the simulation period. Unit are in metres (depth).')
      t2 = nc.variables[var_name][0,:,:]
      #t2 = nc.variables[var_name][day,:,:]
      #for d in range(day, daySum):
      print('number of days or months', len(nc.variables[var_name][:,0,0]))
      for d in range(len(nc.variables[var_name][:,0,0])):
        t2 += nc.variables[var_name][d,:,:]
    else:
      t2 = nc.variables[var_name][day,:,:]
    
    df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
    for i in range(len(t2[:,0])):
      for j in range(len(t2[0,:])):

        if t2[i,j]!=1200:
          df['Latitude'].append(lat[i])
          df['Longitude'].append(lon[j])

          if var_name == 'head_development' and invert == True:
            df[var_name].append(100-t2[i,j])
          else:
            #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
            df[var_name].append(t2[i,j])

          #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
          #df[var_name].append(t2[i,j]) #Notice the minimum
          df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
          #df_yield['Yield'].append(t2[i,j])

    if output==outputFolder:
        difference = df[var_name].copy()
     
    elif calcDifference == False:
        print('not in difference')
        difference = [difference[i] +  df[var_name][i] for i in range(len(difference))]
    
    elif calcDifference == True:
        print('in real difference')
        difference = [difference[i] - df[var_name][i] for i in range(len(difference))]

  df['difference'] = difference
  if compare==True:
    var_name = 'difference'


  if colourScale == None:
    colourScale="YlGnBu"

  if var_name == 'discharge_monthtot':
    #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
 
  else:
    fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z='difference', radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=7, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

  # Bhima zoom = 6.9
  fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

  fig.update_layout(
      title_text = var_name,
      margin=dict(l=5, r=5, t=30, b=10),
      paper_bgcolor='rgba(0,0,0,0)',
      font=dict( color="white"))

  if dark != True:
      fig.update_layout(
          font=dict( color="black"))

  fig.show()

In [4]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map_noTime(var_name, day, day2=None, density=True, dark=True, daySum=None, colourScale=None, Rad=1, invert=False, outputFolder = 'C:\CWatM_output', colourRange=None, compare=None):

      # The function displays the map of *var_name* on *day*.
      # If day2 != None, this displayed the difference between *day* and *day2*.
    nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')
    lat = nc.variables['lat'][:]
    lon = nc.variables['lon'][:]
    print(len(lon))
    t2 = nc.variables[var_name][:,:]

    df = {'Latitude':[], 'Longitude':[], var_name:[]}
    for i in range(len(t2[:,0])):
        for j in range(len(t2[0,:])):
            if t2[i,j]>0:
                df['Latitude'].append(lat[i])
                df['Longitude'].append(lon[j])
                #df[var_name].append(min(500000000,t2[i,j]))
                df[var_name].append(t2[i,j])
                #df[var_name].append(1)

    if colourScale == None:
        colourScale="YlGnBu"

    if density:
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    else:
        fig = px.scatter_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', size=var_name, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = [0,300], color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

      # Bhima zoom = 6.9
    fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

    fig.update_layout(
        title_text = var_name,
        margin=dict(l=5, r=5, t=30, b=10),
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict( color="white"))


    if dark != True:
        fig.update_layout(font=dict( color="black"))

    fig.show()

In [5]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map_noTimeXY(var_name, day, day2=None, density=True, dark=True, daySum=None, colourScale=None, Rad=1, invert=False, outputFolder = 'C:\CWatM_output', colourRange=None, compare=None):

      # The function displays the map of *var_name* on *day*.
      # If day2 != None, this displayed the difference between *day* and *day2*.
    nc = NetCDFFile(outputFolder + '/'+var_name + '.nc')
    lat = nc.variables['y'][:]
    lon = nc.variables['x'][:]
    print(len(lon))
    t2 = nc.variables[var_name][:,:]

    df = {'Latitude':[], 'Longitude':[], var_name:[]}
    for i in range(len(t2[:,0])):
        for j in range(len(t2[0,:])):
            if t2[i,j]>0:
                df['Latitude'].append(lat[i])
                df['Longitude'].append(lon[j])
                df[var_name].append(min(500000000,t2[i,j]))
                #df[var_name].append(t2[i,j])
                #df[var_name].append(1)

    if colourScale == None:
        colourScale="YlGnBu"

    if density:
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
    else:
        fig = px.scatter_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', size=var_name, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = [0,300], color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

      # Bhima zoom = 6.9
    fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

    fig.update_layout(
        title_text = var_name,
        margin=dict(l=5, r=5, t=30, b=10),
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict( color="white"))


    if dark != True:
        fig.update_layout(font=dict( color="black"))

    fig.show()

In [10]:
from pyproj import Proj, transform
Y = [2766500.0, 2765500.0, 2764500.0, 2763500.0, 2762500.0, 2761500.0, 2760500.0, 2759500.0, 2758500.0, 2757500.0, 2756500.0, 2755500.0, 2754500.0, 2753500.0, 2752500.0, 2751500.0, 2750500.0, 2749500.0, 2748500.0, 2747500.0, 2746500.0, 2745500.0, 2744500.0, 2743500.0, 2742500.0, 2741500.0, 2740500.0, 2739500.0, 2738500.0, 2737500.0, 2736500.0, 2735500.0, 2734500.0, 2733500.0, 2732500.0, 2731500.0, 2730500.0, 2729500.0, 2728500.0, 2727500.0, 2726500.0, 2725500.0, 2724500.0, 2723500.0, 2722500.0, 2721500.0, 2720500.0, 2719500.0, 2718500.0, 2717500.0, 2716500.0, 2715500.0, 2714500.0, 2713500.0, 2712500.0, 2711500.0, 2710500.0, 2709500.0, 2708500.0, 2707500.0, 2706500.0, 2705500.0, 2704500.0, 2703500.0, 2702500.0, 2701500.0, 2700500.0, 2699500.0, 2698500.0, 2697500.0, 2696500.0, 2695500.0, 2694500.0, 2693500.0, 2692500.0, 2691500.0, 2690500.0, 2689500.0, 2688500.0, 2687500.0, 2686500.0, 2685500.0, 2684500.0, 2683500.0, 2682500.0, 2681500.0, 2680500.0, 2679500.0, 2678500.0, 2677500.0, 2676500.0, 2675500.0, 2674500.0, 2673500.0, 2672500.0, 2671500.0, 2670500.0, 2669500.0, 2668500.0, 2667500.0, 2666500.0, 2665500.0, 2664500.0, 2663500.0, 2662500.0, 2661500.0]
X = [4478500.0, 4479500.0, 4480500.0, 4481500.0, 4482500.0, 4483500.0, 4484500.0, 4485500.0, 4486500.0, 4487500.0, 4488500.0, 4489500.0, 4490500.0, 4491500.0, 4492500.0, 4493500.0, 4494500.0, 4495500.0, 4496500.0, 4497500.0, 4498500.0, 4499500.0, 4500500.0, 4501500.0, 4502500.0, 4503500.0, 4504500.0, 4505500.0, 4506500.0, 4507500.0, 4508500.0, 4509500.0, 4510500.0, 4511500.0, 4512500.0, 4513500.0, 4514500.0, 4515500.0, 4516500.0, 4517500.0, 4518500.0, 4519500.0, 4520500.0, 4521500.0, 4522500.0, 4523500.0, 4524500.0, 4525500.0, 4526500.0, 4527500.0, 4528500.0, 4529500.0, 4530500.0, 4531500.0, 4532500.0, 4533500.0, 4534500.0, 4535500.0, 4536500.0, 4537500.0, 4538500.0, 4539500.0, 4540500.0, 4541500.0, 4542500.0, 4543500.0, 4544500.0, 4545500.0, 4546500.0, 4547500.0, 4548500.0, 4549500.0, 4550500.0, 4551500.0, 4552500.0, 4553500.0, 4554500.0, 4555500.0, 4556500.0, 4557500.0, 4558500.0, 4559500.0, 4560500.0, 4561500.0, 4562500.0, 4563500.0, 4564500.0, 4565500.0, 4566500.0, 4567500.0, 4568500.0, 4569500.0, 4570500.0, 4571500.0, 4572500.0, 4573500.0, 4574500.0, 4575500.0, 4576500.0, 4577500.0, 4578500.0, 4579500.0, 4580500.0, 4581500.0, 4582500.0, 4583500.0, 4584500.0, 4585500.0, 4586500.0]

inProj = Proj(init='epsg:4258')
outProj = Proj(init='epsg:4326')
#x1,y1 = -11705274.6374,4826473.6922
#x2,y2 = transform(inProj,outProj,x1,y1)
#print x2,y2

c:\users\nb-smilovic\envs\iiasa\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\users\nb-smilovic\envs\iiasa\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\users\nb-smilovic\envs\iiasa\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-c

In [59]:
import pandas as pd
import plotly.express as px

# np.minimum(np.maximum(df[var_name], 1),10)
def display_map(var_name, day, day2=None, dark=True, daySum=None, colourScale=None, Rad=5, invert=False, outputFolder = 'C:\CWatM_output', colourRange=None, compare=None):

      # The function displays the map of *var_name* on *day*.
      # If day2 != None, this displayed the difference between *day* and *day2*.

      print(var_name)
      if 'totalend' in var_name or 'annual' in var_name:
            nc = NetCDFFile(outputFolder + '\\'+var_name + '.nc')
      elif 'month' not in var_name:
      #if var_name == 'discharge' or var_name == 'head_development' or var_name == 'act_nonIrrWithdrawalGW' or var_name=='totalET' or var_name == 'act_nonIrrWithdrawalSW':
        nc = NetCDFFile(outputFolder + '\\'+var_name +'_daily.nc')
      else:
        nc = NetCDFFile(outputFolder + '\\'+var_name + '.nc')
      lat = []
      lon = []
      _lat = nc.variables['y'][:]
      _lat = [i for i in _lat]
      _lon = nc.variables['x'][:]
      _lon = [i for i in _lon]
      time = nc.variables['time'][:]
      print(_lat)
      print(_lon)
      for i,j in (_lon, _lat):
            print(i, j)
      print(len(_lat))
      for l in range(len(_lat)):
            print(_lon(l))
            x2,y2 = transform(inProj,outProj,_lon(l),_lat(l))
            
            lat.extend(y2)
            lon.extend(x2)
    
      print(lat)
      if day2 != None:
        print('Showing difference between timestep '+str(day)+ ' and timestep'+str(day2))
        t2 = nc.variables[var_name][day,:,:] - nc.variables[var_name][day2,:,:]
      elif daySum != None: 
        print('Showing the sum of '+var_name+ ' over the simulation period.')
        t2 = nc.variables[var_name][0,:,:]
        #t2 = nc.variables[var_name][day,:,:]
        #for d in range(day, daySum):
        print('number of days', len(nc.variables[var_name][:,0,0]))
        for d in range(len(nc.variables[var_name][:,0,0])):
          t2 += nc.variables[var_name][d,:,:]
      else:
        t2 = nc.variables[var_name][day,:,:]

      df = {'Latitude':[], 'Longitude':[], var_name:[], 'radius':[]}
      for i in range(len(t2[:,0])):
        for j in range(len(t2[0,:])):

          if t2[i,j]>0:
            df['Latitude'].append(lat[i])
            df['Longitude'].append(lon[j])

            if var_name == 'head_development' and invert == True:
              df[var_name].append(100-t2[i,j])
            else:
              #df[var_name].append(min(1200,t2[i,j])) #Notice the minimum
              df[var_name].append(t2[i,j])

            #df[var_name].append(max(min(1200,t2[i,j]),0.3)) #Notice the minimum
            #df[var_name].append(t2[i,j]) #Notice the minimum
            df['radius'].append(np.maximum(1, t2[i,j]/100)) #Notice the minimum
            #df_yield['Yield'].append(t2[i,j])
      if colourScale == None:
        colourScale="YlGnBu"

      if var_name == 'discharge_monthtot':
        #fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.23, lon=74.8), range_color = [0,300], zoom=8.5, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), range_color = [0,300], zoom=8.8, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 
        
      else:
        fig = px.density_mapbox(pd.DataFrame(df), lat='Latitude', lon='Longitude', z=var_name, radius=Rad, center=dict(lat=18.1375, lon=74.4625), zoom=6.9, range_color = colourRange, color_continuous_scale=colourScale, mapbox_style="stamen-terrain") 

      # Bhima zoom = 6.9
      fig.update_layout(mapbox_style="satellite", mapbox_accesstoken='pk.eyJ1Ijoic21pbG92aWMiLCJhIjoiY2tjamVwYWF2MThncTJ4cXFxbWVjdzY2diJ9.u_haNh8K1p-9oATjHgQ6LA')

      fig.update_layout(
          title_text = var_name,
          margin=dict(l=5, r=5, t=30, b=10),
          paper_bgcolor='rgba(0,0,0,0)',
          font=dict( color="white"))


      if dark != True:
          fig.update_layout(
              font=dict( color="black"))

      fig.show()

In [56]:
#display_map_noTime('head_development',  200, colourScale='Blues', Rad=5, dark=False, daySum = True, outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_noTime('frac_totalIrr_max_totalend',  -1, density=True, colourScale="agsunset", Rad=3, dark=False, daySum = True, outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('head_development_monthend', day=22, Rad=2.1,  colourScale ='RdYlBu_r',dark=False, colourRange= [0,20], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_sum('fracVegCover[3]_monthend', 'fracVegCover[1]_monthend', day=1, Rad=2, colourScale ='BrBG', dark=False,outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('fracVegCover[3]_monthend, day=-1, Rad=2.1,  colourScale ='RdYlBu_r',dark=False, colourRange= [0,20], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map('discharge_annualtot', day=-1, Rad=2.1,  colourScale ='Blues',dark=False, colourRange= [0,100], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'
#display_map_sum('fracVegCover[3]_monthend', 'fracVegCover[1]_monthend', day=1, difference = True, Rad=2, colourScale ='BrBG', dark=False,outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'

#display_map_sum_differentRuns('fracVegCover[3]_monthend', day=-1, calcDifference = True, Rad=2, colourScale ='RdBu_r', dark=False,outputFolder = 'C:\CWatM_output') #)2.6) #colourScale='balance_r'
#display_map_noTime('waterBodyID_totalend',  -1, density=True, colourScale="agsunset", Rad=3, dark=False, daySum = True, colourRange=[0,1], outputFolder = 'C:\CWatM_output_Examples\CWatM_output_06Sept_mswep') #)2.6) #colourScale='balance_r'

#display_map('gwdepth_monthend', day=-1, Rad=16, daySum=False, colourScale ='agsunset', dark = False, outputFolder = r'C:\Data\bhima_Input_mainCWatM\Output')


In [57]:
output_folder = r'P:\watmodel\CWATM\calibration\multi_salzburg\final_run\out_MS'

In [60]:
display_map('discharge', day=-1, Rad=22, daySum=True, colourScale ='Blues', outputFolder = output_folder) #)2.6) #colourScale='balance_r'3

discharge
[2766500.0, 2765500.0, 2764500.0, 2763500.0, 2762500.0, 2761500.0, 2760500.0, 2759500.0, 2758500.0, 2757500.0, 2756500.0, 2755500.0, 2754500.0, 2753500.0, 2752500.0, 2751500.0, 2750500.0, 2749500.0, 2748500.0, 2747500.0, 2746500.0, 2745500.0, 2744500.0, 2743500.0, 2742500.0, 2741500.0, 2740500.0, 2739500.0, 2738500.0, 2737500.0, 2736500.0, 2735500.0, 2734500.0, 2733500.0, 2732500.0, 2731500.0, 2730500.0, 2729500.0, 2728500.0, 2727500.0, 2726500.0, 2725500.0, 2724500.0, 2723500.0, 2722500.0, 2721500.0, 2720500.0, 2719500.0, 2718500.0, 2717500.0, 2716500.0, 2715500.0, 2714500.0, 2713500.0, 2712500.0, 2711500.0, 2710500.0, 2709500.0, 2708500.0, 2707500.0, 2706500.0, 2705500.0, 2704500.0, 2703500.0, 2702500.0, 2701500.0, 2700500.0, 2699500.0, 2698500.0, 2697500.0, 2696500.0, 2695500.0, 2694500.0, 2693500.0, 2692500.0, 2691500.0, 2690500.0, 2689500.0, 2688500.0, 2687500.0, 2686500.0, 2685500.0, 2684500.0, 2683500.0, 2682500.0, 2681500.0, 2680500.0, 2679500.0, 2678500.0, 2677500.0,

ValueError: too many values to unpack (expected 2)